# Installatie

Klik op onderstaande knop (open in Colab) om de notebook te openen. Daarna kun je in Colab het installatiescript draaien. Hiervoor heb je een Google account nodig.

<a target="_blank" href="https://colab.research.google.com/github/brunoruns/pix_llm/blob/main/notebook.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

### Installatiescript
Klik op onderstaande cel om alle te installeren wat we vandaag nodig hebben. Dit wordt NIET geïnstalleerd op je computer, maar op een computer in de cloud.

In [ ]:
import os

!wget -O environment.yml "https://github.com/brunoruns/pix_llm/blob/main/environment.yml"

env_file = "environment.yml"
if not os.path.exists('/usr/local/bin/conda'):
    print("Installing Conda...")
    !wget -c https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O miniconda.sh
    !bash miniconda.sh -b -f -p /usr/local
    !rm miniconda.sh
    !/usr/local/bin/conda init

# Activate conda and install packages from the environment file
!eval "$(conda shell.bash hook)" && conda env create -f {env_file} -p /usr/local/env
# Activate the environment and configure Jupyter to use it
!eval "$(conda shell.bash hook)" && conda activate /usr/local/env && python -m ipykernel install --user --name=colab_env --display-name "Python (colab_env)"

# Restart kernel
import IPython
IPython.display.clear_output()

print("Conda environment installed and ready! Please select 'Python (colab_env)' as the kernel if needed.")

### Instructie

Je kan deze notebook gebruiken door met je cursus in een cel te gaan staan, en dan SHIFT-ENTER te typen. De code in die cel wordt dan uitgevoerd. Wat je ziet is python code, een populaire programmeertaal om met datastructure en AI te werken. De tussenresultaten van de code (de variabelen) worden zolang deze notebook open staat bewaard. Dit wil zeggen dat wanneer je achtereenvolgens volgende cellen gaat uitvoeren, er uiteindelijk 8 zal worden geprint

In [75]:
a = 5

In [76]:
b = 3

In [77]:
c = a + b
print(c)

# Bag of words
Bag-of-Words (BoW) is een eenvoudige techniek om tekst om te zetten in numerieke vorm, maar het heeft een aantal beperkingen, vooral bij het vastleggen van betekenis en context. Bij de one-hot encoding representeren we elk woord als een unieke vector met alleen nullen en een enkele 1 op de positie die overeenkomt met dat woord in de woordenschat.

Bijvoorbeeld, stel we hebben de volgende woordenlijst (vocabulaire):

In [78]:
["koning", "koningin", "man", "vrouw", "troon"]  

Dan zou de one-hot encoding er als volgt uitzien:

In [79]:
koning    = [1, 0, 0, 0, 0]  
koningin  = [0, 1, 0, 0, 0]  
man       = [0, 0, 1, 0, 0]  
vrouw     = [0, 0, 0, 1, 0]  
troon     = [0, 0, 0, 0, 1]  


Het probleem met deze manier van werken is dat er geen context of betekenis aan woorden kunnen worden meegegeven. Koning en koningin staan qua betekenis dicht bij elkaar, maar dat is uit deze representatie niet duidelijk. Bovendien heb je even lange vectoren nodig als het aantal woorden in je woordenlijst: dat is niet schaalbaar voor echte talen. Het Algemeen Nederlands Woordenboek (https://anw.ivdnt.org/search) bevat bijvoorbeeld meer dan 290 000 woorden.

# Embeddings
Word2Vec lost deze problemen op door woordvectoren te leren die in een lagere dimensie liggen (bijv. 300 getallen i.p.v. 100.000) én die betekenisvolle relaties tussen woorden behouden. Elk woord wordt dan 'ingebed' in een 300-dimensionele vectorruimte, vandaar de naam 'embedding'.

https://medium.com/@manansuri/a-dummys-guide-to-word2vec-456444f3c673

In [80]:
# de code hieronder laadt bestaande software in uit zogenaamde software bibliotheken (libraries)
import gensim.downloader as api # dit gaan we gebruiken om een groot stuk voorbeeldtekst te downloaden

In [81]:
corpus = api.load('text8')  # download het corpus en slaat dit op in een variable corpus als een lijst

Dit bevat data, grotendeels afkomstig van Wikipedia. De eerste paar items van dit corpus zien er als volgt uit:

In [82]:
list(corpus)

Het corpus bestaat uit een lijst, die zelf telkens bestaat uit meerdere lijsten (lijst in lijst). Met de len-functie en een FOR-loop kun je opvragen hoeveel items er in deze lijsten zitten, door over elk van de lijstjes in de grote lijst te lopen:

In [83]:
grote_lijst = list(corpus)
total_words = 0
for kleine_lijst in grote_lijst:
    print("Het eerste woord van deze lijst is ", kleine_lijst[0], ", het aantal woorden in deze lijst is ", len(kleine_lijst))
    total_words = total_words + len(kleine_lijst)
print("Het totaal aantal woorden in dit corpus is ", total_words)

We weten nu hoeveel woorden er in totaal in deze tekst zitten. Dit geeft een idee van de grootte van deze dataset.

## Word2Vec

### Probleemstelling

We willen idealiter een vector die bepaalde aspecten van een woord kan capteren. Bekijk volgend fictief voorbeeld:

![Source:https://medium.com/@manansuri/a-dummys-guide-to-word2vec-456444f3c673](https://miro.medium.com/v2/resize:fit:720/format:webp/1*Z-EOcLtUDlhvVuaruyM8EQ.png)

Links staan kenmerken zoals 'koninklijkheid', 'mannelijkheid', 'vrouwelijkheid' en 'leeftijd'. Op die manier zouden we voor de 5 voorbeeldwoorden in de kolommen getallen kunnen kleven die 'coderen' in welke mate dat woord die kenmerken heeft. Wij kunnen dat omdat wij de taal spreken, een computer moet dit doen via een omweg. Een computer kijkt naar welke woorden in een bepaalde zin 'inpassen'.

In de zin "de ____ zit op de troon" passen typisch een aantal woorden:
- koning
- koningin
- prins

Deze zin geeft dus een context waar sommige woorden in passen, en andere niet. Word2Vec buit dit idee uit, door voor een gegeven tekst, de context rond elk woord te bekijken (de woorden vlak ervoor en vlak erna) en te kijken welke woorden in dezelfde contexten kunnen voorkomen. We hebben hierboven een groot corpus ingeladen, en dat gaan we gebruiken om het model te trainen - het model gaat op basis van die tekst zelf een zo goed mogelijke inbedding creëren.

Je zou nu alle combinaties van woorden als verschillende contexten kunnen maken en daar een tabel van laten opstellen. Maar zelfs met een computer, is dat veel te veel werk, omdat het aantal mogelijke combinaties enorm snel stijgt. Er wordt iets slimmer gedaan, maar we zullen het model eerst gebruiken.

In [84]:
# Word2Vec is een model dat in 2013 werd ontwikkeld door onderzoekers van Google
from gensim.models.word2vec import Word2Vec # dit bevat een gekende embedder
model = Word2Vec(sentences=corpus, vector_size=50, window=5, min_count=1, workers=4) # op deze stap wordt het model ingeladen én getrained op het corpus. Daarom duurt dit wel even.

Nu het model 'geleerd' heeft wat het moet doen, kan je aan het model vragen om van een woord de vector-embedding weer te geven. Zoals verwacht, is dit een vector met lengte 50.

In [85]:
# Toon embedding van een woord
print(model.wv["computer"])  # Vector van het woord "computer"

In wat volgt kan je nu 'met woorden rekenen'. We zullen een specifieke berekening doen:

In [86]:
king_vector = model.wv["king"]
man_vector = model.wv["man"]
woman_vector = model.wv["woman"]
queen_vector = model.wv['queen']

new_vector = king_vector - man_vector + woman_vector
print(new_vector)

Zo dadelijk zullen we zien dat deze vector, hoewel hij op het eerste zicht helemaal niet begrijpelijk lijkt, goed geslaagd is in zijn opzet.

![Word Embedding Analogy](https://www.researchgate.net/profile/Jayaraman-J-Thiagarajan-2/publication/319370400/figure/fig1/AS:639487218548736@1529477042252/n-the-word-embedding-space-the-analogy-pairs-exhibit-interesting-algebraic-relationships.png)


### Hoe is dit model gemaakt ?
Je kan dit model voorstel als een opeenvolging van input naar output.

![1_tmyks7pjdwxODh5-gL3FHQ.webp](https://miro.medium.com/v2/resize:fit:720/format:webp/1*XBaqjqpnBIXtXLzWjkuLQQ.png)

Als input is elk woord gecodeerd volgens zijn positie in de tekst. De oranje cellen hierboven spelen elk de rol van een dimensie van de uiteindelijke vectorruimte waarnaar we de data willen omzetten. In dit geval 300. De output cellen in rood, geven aan of dit woord overeenkomt met een aantal op voorhand gedefinieerde woorden. Dit is een voorbeeld van een neuraal netwerk.

Link naar een voorbeeld: https://playground.tensorflow.org 

Tussen elke laag in een neuraal netwerk zijn er pijltjes die aangeven waar elke output opnieuw als input wordt gebruikt. Elk van deze pijltjes heeft een bepaald gewicht, dat het neuraal network zelf moet leren. Vergelijk het met een pad in je brein waarin bepaalde neuronen sterker worden geactiveerd bij bepaalde input. Tussen de oranje en de rode laag wordt een softmax classifier gebruikt; die numerieke waarden omzet in kansen. Zo wordt een input vector omgezet in een reeks output waardes, waarbij de output waardes aangeven hoe groot de kans is dat een willekeurig gekozen woord in de buurt komt van een ander woord.

Om te begrijpen hoe dit werkt, kijken we naar een heel kleine versie van een neuraal netwerk:

In [87]:
import networkx as nx
import matplotlib.pyplot as plt

def draw_neural_network():
    """
    Teken een diagram van een klein neuraal netwerk met:
    - 2 input neuronen
    - 4 neuronen in verborgen laag 1
    - 3 neuronen in verborgen laag 2
    - 1 output neuron
    """
    # 🔹 Netwerkstructuur
    layers = [2, 4, 3, 1]  # Aantal neuronen per laag
    layer_positions = [0, 1, 2, 3]  # X-posities per laag
    
    # 🔹 Maak een lege graaf (graph)
    G = nx.DiGraph()

    # 🔹 Posities van de neuronen bepalen
    pos = {}  
    y_offsets = {}  # Houd bij hoeveel neuronen er al zijn in een laag

    for layer_idx, num_nodes in enumerate(layers):
        y_offset = -(num_nodes - 1) / 2  # Zorgt ervoor dat neuronen netjes verdeeld zijn
        
        for node_idx in range(num_nodes):
            node_name = f"L{layer_idx}_N{node_idx}"  # Unieke naam per neuron
            G.add_node(node_name, layer=layer_idx)  # Voeg neuron toe
            pos[node_name] = (layer_positions[layer_idx], y_offset + node_idx)  # Bepaal positie

    # 🔹 Verbind neuronen tussen de lagen
    for layer_idx in range(len(layers) - 1):
        for src_idx in range(layers[layer_idx]):
            for dest_idx in range(layers[layer_idx + 1]):
                src_node = f"L{layer_idx}_N{src_idx}"
                dest_node = f"L{layer_idx + 1}_N{dest_idx}"
                G.add_edge(src_node, dest_node)  # Voeg verbinding toe

    # 🔹 Visualisatie van het neurale netwerk
    plt.figure(figsize=(8, 6))
    nx.draw(G, pos, with_labels=False, node_size=800, node_color="skyblue", edge_color="gray", width=1.5, arrows=False)

    # Labels per laag
    for layer_idx, num_nodes in enumerate(layers):
        x_pos = layer_positions[layer_idx]
        y_pos = sum([pos[f"L{layer_idx}_N{i}"][1] for i in range(num_nodes)]) / num_nodes
        label = "Input" if layer_idx == 0 else ("Output" if layer_idx == len(layers) - 1 else f"Hidden Layer {layer_idx}")
        plt.text(x_pos, y_pos + 0.8, label, fontsize=12, ha="center", color="black", fontweight="bold")

    plt.title("Small Neural Netwerk Layout")
    plt.show()

# 🚀 Genereer en toon de netwerkvisualisatie
draw_neural_network()


In [88]:
import numpy as np

# activatie functies
def relu(x):
    return np.maximum(0, x)  # ReLU activatie

def sigmoid(x):
    return 1 / (1 + np.exp(-x))  # Sigmoid activatie

def id(x): # dit is de identiteitsfunctie, die 'doet niks'
    return x

# Blauwdruk van een klein neural netwerk
class SmallNeuralNetwork:
    def __init__(self):
        np.random.seed(42)  # om te zorgen dat de willekeurige getallen identiek blijven
        
        # gewichten en biases
        self.W1 = np.random.randn(2, 4)  # Input Layer (2) → Hidden Layer 1 (4)
        self.b1 = np.random.randn(4)

        self.W2 = np.random.randn(4, 3)  # Hidden Layer 1 (4) → Hidden Layer 2 (3)
        self.b2 = np.random.randn(3)

        self.W3 = np.random.randn(3, 1)  # Hidden Layer 2 (3) → Output Layer (1)
        self.b3 = np.random.randn(1)

    def forward(self, x):
        """
        Inferentie uitvoeren
        :param x: Input vector of shape (2,)
        :return: Output scalar (prediction)
        """
        # Ensure input is 2D (batch size 1)
        x = x.reshape(1, -1)

        # 🚀 Forward Propagation: Matrix Multiplication
        self.z1 = np.dot(x, self.W1) + self.b1  # Input → Hidden 1
        self.a1 = relu(self.z1)                 # Apply ReLU
        
        self.z2 = np.dot(self.a1, self.W2) + self.b2  # Hidden 1 → Hidden 2
        self.a2 = relu(self.z2)                       # Apply ReLU
        
        self.z3 = np.dot(self.a2, self.W3) + self.b3  # Hidden 2 → Output
        output = sigmoid(self.z3)                     # Apply Sigmoid
        
        return output.squeeze()  # Convert to scalar

# Initialisatie
nn = SmallNeuralNetwork()

# Voorbeeldgebruik
input_vector = np.array([0.5, -1.5])

# 🔍 Run the Forward Pass
output = nn.forward(input_vector)

# Resultaten printen
print("Input Vector:", input_vector)
print("Predicted Output:", output)


Je kan zo'n netwerk trainen door een *loss-function* op te stellen. Dat is een functie die 'meet' in welke mate de output van je netwerk aan bepaalde eigenschappen voldoet. Voor bovenstaan netwerk is dit een relalistische loss-functie:

In [89]:
def binary_cross_entropy(y_true, y_pred):
    return -np.mean(y_true * np.log(y_pred + 1e-8) + (1 - y_true) * np.log(1 - y_pred + 1e-8))

def binary_cross_entropy_derivative(y_true, y_pred):
    return -(y_true / (y_pred + 1e-8)) + ((1 - y_true) / (1 - y_pred + 1e-8))

Je kan dan, als je bepaalde voorbeelden hebt waarvan je de gewenste uitkomst kent, volgende strategie volgen:
- een voorbeeldvector invoeren
- de voorspelde uitkomst (y_predicted) en de gekende uitkomst van het voorbeeld (y_true) vergelijken met de binary_cross_entropy
- in de functie binary_cross_entropy, kijken in welke richting de functie een betere score heeft
- de gewichtjes van het neuraal netwerk aanpassen zodat de voorspelling dichter bij de voorbeelduitkomst zit

Dit proces heet backtracking. Centraal in het algoritme zit een techniek (gradient descent) die de afgeleide van de functies nodig heeft om te kunnen bepalen in welke richting de gewichtjes moeten worden aangepast. Voor de volledigheid geef ik hieronder een implentatie van hoe het model hierboven dan eerst kan worden getraind, als je voorbeeldinformatie hebt.

In [90]:
import numpy as np

# 🚀 Activatiefuncties en hun afgeleiden
def relu(x):
    return np.maximum(0, x)  # ReLU activatie: negatieve waarden worden 0

def relu_afgeleide(x):
    return (x > 0).astype(float)  # Afgeleide van ReLU: 1 als x > 0, anders 0

def sigmoid(x):
    return 1 / (1 + np.exp(-x))  # Sigmoid activatie: output tussen 0 en 1

def sigmoid_afgeleide(x):
    s = sigmoid(x)
    return s * (1 - s)  # Afgeleide van sigmoid

# Loss functie: Binair Cross-Entropy
def binary_cross_entropy(y_true, y_pred):
    return -np.mean(y_true * np.log(y_pred + 1e-8) + (1 - y_true) * np.log(1 - y_pred + 1e-8))

def binary_cross_entropy_afgeleide(y_true, y_pred):
    return -(y_true / (y_pred + 1e-8)) + ((1 - y_true) / (1 - y_pred + 1e-8))

# Klein neuraal netwerk met backpropagation
class KleinNeuraalNetwerk:
    def __init__(self, leer_snelheid=0.05):
        np.random.seed(42)  # Zorgen voor reproduceerbare resultaten
        self.lr = leer_snelheid  # Leersnelheid voor gradient descent
        
        # Initialisatie van gewichten en biases
        self.W1 = np.random.randn(2, 4) * 0.01 # Input → Verborgen laag 1 (4 neuronen)
        self.b1 = np.zeros(4)

        self.W2 = np.random.randn(4, 3) *0.01 # Verborgen laag 1 → Verborgen laag 2 (3 neuronen)
        self.b2 = np.zeros(3)

        self.W3 = np.random.randn(3, 1) * 0.01 # Verborgen laag 2 → Output (1 neuron)
        self.b3 = np.zeros(1)

    def forward(self, x):
        """
        Voer de forward pass uit met matrixvermenigvuldiging.
        :param x: Input vector met vorm (2,)
        :return: Output waarde (voorspelling)
        """
        x = x.reshape(1, -1)  # Zorgen dat input tweedimensionaal is
        
        # 🚀 Forward propagatie: matrixvermenigvuldiging en activaties
        self.z1 = np.dot(x, self.W1) + self.b1
        self.a1 = relu(self.z1)  # Toepassen van ReLU
        
        self.z2 = np.dot(self.a1, self.W2) + self.b2
        self.a2 = relu(self.z2)  # Toepassen van ReLU
        
        self.z3 = np.dot(self.a2, self.W3) + self.b3
        self.a3 = sigmoid(self.z3)  # Toepassen van sigmoid (laatste output)

        return self.a3.squeeze()  # Omzetten naar een scalaire waarde

    def backward(self, x, y_true):
        """
        Voer backpropagation uit om gewichten te updaten.
        :param x: Input vector (2,)
        :param y_true: Werkelijke labelwaarde (0 of 1)
        """
        x = x.reshape(1, -1)  # Zorgen dat input tweedimensionaal is
        
        # 🚀 Forward pass opnieuw uitvoeren om de activaties te berekenen
        self.forward(x)

        # 🚀 Berekening van de afgeleide van de verliesfunctie
        dL_dz3 = binary_cross_entropy_afgeleide(y_true, self.a3) * sigmoid_afgeleide(self.z3)
        #dL_dz3 = (self.a3 - y_true)

        # 🚀 Backpropagation door de lagen
        dL_dW3 = np.dot(self.a2.T, dL_dz3)
        dL_db3 = dL_dz3

        dL_dz2 = np.dot(dL_dz3, self.W3.T) * relu_afgeleide(self.z2)
        dL_dW2 = np.dot(self.a1.T, dL_dz2)
        dL_db2 = dL_dz2

        dL_dz1 = np.dot(dL_dz2, self.W2.T) * relu_afgeleide(self.z1)
        dL_dW1 = np.dot(x.T, dL_dz1)
        dL_db1 = dL_dz1

        # 🚀 Gewichten en biases updaten met gradient descent
        self.W3 -= self.lr * dL_dW3
        self.b3 -= self.lr * np.sum(dL_db3, axis=0)

        self.W2 -= self.lr * dL_dW2
        self.b2 -= self.lr * np.sum(dL_db2, axis=0)

        self.W1 -= self.lr * dL_dW1
        self.b1 -= self.lr * np.sum(dL_db1, axis=0)

    def train(self, X, y, epochs=1000):
        """
        Train het netwerk met backpropagation.
        :param X: Trainingsdata (N x 2)
        :param y: Doellabels (N x 1)
        :param epochs: Aantal iteraties
        """
        for epoch in range(epochs):
            verlies = 0
            for i in range(len(X)):
                self.backward(X[i], y[i])  # Backpropagation uitvoeren
                verlies += binary_cross_entropy(y[i], self.forward(X[i]))  # Verlies berekenen
            
            if epoch % 100 == 0:
                print(f"Epoch {epoch}, Verlies: {verlies / len(X):.4f}")

# Het neurale netwerk trainen
nn = KleinNeuraalNetwerk(leer_snelheid=0.1)

# Voorbeeld trainingsdata (XOR-probleem)
X_train = np.array([
    [0, 0],
    [0, 1],
    [1, 0],
    [1, 1]
])

y_train = np.array([0, 1, 1, 0])  # Binaire labels

# Train het netwerk
nn.train(X_train, y_train, epochs=1000)

# Voorspellingen testen
for x in X_train:
    print(f"Input: {x}, Voorspelling: {nn.forward(x):.4f}")


Toegepast op het Word2Vec neuraal netwerk: je kan, als je een hele tekst hebt, de woorden en hun context stukje per stukje aan dit model geven, en de gewichtjes laten verbeteren zodanig dat uiteindelijk dit model zo goed mogelijk deze waardes benaderd.
- We hebben namelijk veel voorbeelden van woorden en de omliggende woorden die voorspeld zouden moeten worden
- we kunnen dus dit model *trainen*: we geven achtereenvolgens een woord en de verwachte uitkomst, en laten het model de gewichtjes aanpassen om zo dicht mogelijk bij het eindresultaat te komen.
- op die manier gaat de computer zélf leren welke informatie er in die vectorrepresentatie wordt bijgehouden, dit zal voor een mens niet meer zo makkelijk interpreteerbaar zijn als "koninklijkheid" of "leeftijd" zoals voordien.

Het enige wat daarbij nog extra kan worden opgemerkt is dat je in het geval van een taalmodel nooit 1 enkele oplossing (woord) hebt voor een bepaalde context (een gemaskeerde zin). Daarom train je het model om een kanstheoretische verdeling als output te geven, die aangeeft wat de kans is dat je een andere woord uit het corpus daar in de buurt tegenkomt. Als je dat op een heel kleine schaal bekijkt:

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from gensim.models import Word2Vec

# 🚀 1. Voorbeeldcorpus: Eenvoudige zinnen met context
corpus_klein = [
    ["hond", "blaffen", "kat", "miauwen", "aaien"],
    ["kat", "aaien", "zacht", "vriendelijk"],
    ["hond", "rennen", "spelen", "bos"],
    ["kat", "spinnen", "slaap", "bank"],
    ["aaien", "zacht", "liefdevol", "kat"],
    ["kind", "aaien", "kat", "vriendelijk"]
]

# Train een Word2Vec-model met 5 dimensies
model_klein = Word2Vec(sentences=corpus_klein, vector_size=2, window=2, min_count=1, workers=4)

# Visualiseer de woordvectoren in 2D
words = ["kat", "aaien", "hond", "blaffen", "zacht", "vriendelijk"]
word_vectors = np.array([model_klein.wv[word] for word in words])

# Plot de woordvectoren
plt.figure(figsize=(6,5))
plt.scatter(word_vectors[:, 0], word_vectors[:, 1])
for word, (x, y) in zip(words, word_vectors):
    plt.text(x, y, word, fontsize=12)
plt.title("Woordvectoren in 2D")
plt.xlabel("Component 1")
plt.ylabel("Component 2")
plt.grid()
plt.show()


![source:](https://images.contentstack.io/v3/assets/bltac01ee6daa3a1e14/blte5e1674e3883fab3/65ef8ba4039fdd4df8335b7c/img_blog_image1_inline_(2).png?width=2048&disable=upscale&auto=webp)

### Wat kun je met het getraind model doen ?
Het getraind model heeft *(op magische wijze!)* een vorm van betekenis geleerd: woorden die 'gelijkaardig' zijn in betekenis, liggen 'dicht bij elkaar. Dat dicht bij elkaar kunnen we uitrekenen:

In [92]:
similarity_score = model.wv.similarity('computer', 'graphics')
print(similarity_score)

Bovendien komt 'dicht bij mekaar' echt overeen met wat je in 2D of 3D al kent voor meetkundige hoeken:

In [93]:
from sklearn.metrics.pairwise import cosine_similarity

In [94]:
new_vec_col = new_vector.reshape(1,-1) # wisselen van rijvector naar kolomvector
queen_vec_col = queen_vector.reshape(1,-1)

In [95]:
similarity_score = cosine_similarity(new_vec_col, queen_vec_col)
similarity_score

In [96]:
import numpy as np

def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2) # dit vermenigvuldigt elementsgewijs en telt alles daarna op. Geeft een scalair als resultaat
    
    # de vectoren moeten worden genormeerd
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    
    # Similarity
    similarity = dot_product / (norm_vec1 * norm_vec2)
    
    return similarity


# je eigen functie gebruiken:
similarity_score = cosine_similarity(new_vector, queen_vector)

print(f"Cosine Similarity: {similarity_score:.6f}")


![cosine similarity](https://miro.medium.com/v2/resize:fit:1100/format:webp/1*GK56xmDIWtNQAD_jnBIt2g.png)

# Attention is all you need
Deep learning models hadden op zich nog niet genoeg kracht om vlot tekst te kunnen verwerken. Het zou duren tot een nieuwe innovatie in 2016 hier verandering in bracht, door in de architectuur van een neuraal netwerk een vorm van geheugen in te bouwen. Dit noemen we 'attention'.

## Positional awareness

In de code hieronder wordt getoond hoe je de positie van een woord in een zin kan coderen.

In [100]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
model_small = Word2Vec(sentences=corpus, vector_size=10, window=4, min_count=1, workers=4)
sentence = ["the", "queen", "does", "not", "like", "art", 'but', 'the', 'king', 'does']
embedding_dim = model_small.vector_size
print(embedding_dim)

In [102]:
word_vectors = torch.tensor([model_small.wv[word] for word in sentence])  # Shape: (sequence_length, embedding_dim)
print(word_vectors)

In [103]:
# Add batch dimension: (1, sequence_length, embedding_dim)
input_sequence = word_vectors.unsqueeze(0)
print(input_sequence)

Hieronder een simpel neuraal netwerk, zonder woordvolgorde:

In [104]:
# Simpel  Neural Network (zonder positie)
class SimpleNN(nn.Module):
    def __init__(self, embedding_dim):
        super(SimpleNN, self).__init__()
        self.fc = nn.Linear(embedding_dim, embedding_dim)  # Transformeer input embeddings

    def forward(self, x):
        return self.fc(x)
    
simple_nn = SimpleNN(embedding_dim=embedding_dim)
output_simple = simple_nn(input_sequence)
print("Simple Neural Network Output (heeft geen notie van positie):\n", output_simple.detach().numpy())

In [105]:
# Transformer met positional encoding
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=10):
        super(PositionalEncoding, self).__init__()

        # Matrix met shape (vorm) (max_len, d_model)
        position = torch.arange(max_len).unsqueeze(1)  # Shape (max_len, 1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * -(np.log(10000.0) / d_model))

        pe = torch.zeros(max_len, d_model)
        pe[:, 0::2] = torch.sin(position * div_term)  # Even indices
        pe[:, 1::2] = torch.cos(position * div_term)  # Oneven indices

        self.pe = pe.unsqueeze(0)  # Shape (1, max_len, d_model)

    def forward(self, x):
        return x + self.pe[:, :x.size(1), :]

# Transformer-achtig Model met positional encoding
class TransformerModel(nn.Module):
    def __init__(self, d_model):
        super(TransformerModel, self).__init__()
        self.positional_encoding = PositionalEncoding(d_model)
        self.fc = nn.Linear(d_model, d_model)  # Transformeer embeddings

    def forward(self, x):
        x = self.positional_encoding(x)
        return self.fc(x)

# Instantiate model
transformer = TransformerModel(d_model=embedding_dim)

# input door model halen
output_transformer = transformer(input_sequence)

print("Simple Neural Network Output (heeft geen notie van positie):\n", output_simple.detach().numpy())
print("\nTransformer met positional encoding output:\n", output_transformer.detach().numpy())


In [106]:
# Visualisatie van het verschil
plt.figure(figsize=(8, 4))
plt.imshow(transformer.positional_encoding.pe.squeeze().numpy(), cmap='viridis', aspect='auto')
plt.colorbar()
plt.title("Positional Encoding Values")
plt.xlabel("Embedding Dimension")
plt.ylabel("Woord positie")

plt.yticks(ticks=range(len(sentence)), labels=sentence)
plt.show()

### Vergelijken van 'gelijkaardige' zinnen

In [107]:
sentences = [
    ["the", "cat", "sat", "on", "the", "mat"],
    ["on", "the", "mat", "sat", "the", "cat"],
    ["the", "mat", "was", "sat", "on", "by", "the", "cat"],
    ["sat", "on", "the", "mat", "the", "cat", "did"]
]

In [108]:
embedding_dim = model_small.vector_size
word_vectors = [torch.tensor([model_small.wv[word] for word in sentence if word in model_small.wv]) for sentence in sentences]
word_vectors = [wv.unsqueeze(0) for wv in word_vectors]

In [109]:
class TransformerWithAttention(nn.Module):
    def __init__(self, d_model):
        super(TransformerWithAttention, self).__init__()
        self.positional_encoding = PositionalEncoding(d_model)
        self.attention = nn.MultiheadAttention(embed_dim=d_model, num_heads=2, batch_first=True)
        self.fc = nn.Linear(d_model, d_model)

    def forward(self, x):
        x = self.positional_encoding(x)
        attn_output, _ = self.attention(x, x, x)  # Self-attention
        return self.fc(attn_output)


In [110]:
import matplotlib.pyplot as plt

# 🚀 Functie om beide modellen in één plot te visualiseren
def plot_sentence_comparison(sentence, simple_embedding, transformer_embedding, sentence_index):
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))  # 1 rij, 2 kolommen

    # Plot voor het simpele model (zonder attention)
    ax1 = axes[0]
    im1 = ax1.imshow(simple_embedding, cmap='viridis', aspect='auto')
    ax1.set_title(f"Simple NN Output (zonder Attention) - Sentence {sentence_index}")
    ax1.set_xlabel("Embedding Dimension")
    ax1.set_ylabel("Word Position")
    ax1.set_yticks(range(len(sentence)))
    ax1.set_yticklabels(sentence)
    fig.colorbar(im1, ax=ax1)

    # Plot voor het transformer model (met attention)
    ax2 = axes[1]
    im2 = ax2.imshow(transformer_embedding, cmap='viridis', aspect='auto')
    ax2.set_title(f"Transformer Output (met Attention) - Sentence {sentence_index}")
    ax2.set_xlabel("Embedding Dimension")
    ax2.set_yticks(range(len(sentence)))
    ax2.set_yticklabels(sentence)
    fig.colorbar(im2, ax=ax2)

    plt.tight_layout()
    plt.show()

# 🚀 Itereer over de zinnen en maak gecombineerde plots
for i, sentence in enumerate(sentences):
    print(f"\nSentence variatie  {i+1}: {' '.join(sentence)}")

    # Visualisatie van beide modellen in één plot
    plot_sentence_comparison(
        sentence,
        outputs_simple_np[i],
        outputs_transformer_np[i],
        sentence_index=i+1
    )


# Voorbij transformers naar GPT

Nu we heel snel hebben uitgelegd wat een transformer is, kunnen we kijken naar GPT modellen, zoals GPT-2. GPT staat voor Generative Pre-trained Transformer, waarvan we de laatste 2 woorden al hebben verklaard:
- Het is een model dat getrained is (op publieke tekst, beschikbaar op het inernet)
- Het is een transformer model, dwz het is een deep neural network met een bepaalde architectuur zodat het belang kan hechten aan woordvolgorde.

Het woord 'generative' betekent dat dit model tijdens het trainen geleerd heeft om in bepaalde stukken tekst een woord aan te vullen, zoals 'De kat krabt de krollen van de ___'. Hierdoor kan dit model, als je het een stuk tekst geeft, het meest waarschijnlijke volgende woord aangeven.

GPT2 is een model dat werd ontwikkeld in 2019, en getraind werd op 8 miljoen webpagina's. We gebruiken hier de kleine versie met 114 miljoen (114 000 000) parameters. Dit wil grosso modo zeggen dat er in het neuraal netwerk 114 000 000 gewichtjes op punt moesten worden gesteld om deze machine te laten werken.

In [111]:
from transformers import pipeline

# We laden dit model in, dit wil zeggen: dit is een versie waar de gewichtjes voor alle duidelijkheid al vast staan (die berekening, het trainen, gebeurde eerder: pre-trained)
generator = pipeline("text-generation", model="gpt2")

In [112]:
# Hierbij een zin
input_text = "We are in a classroom. Artificial Intelligence is changing the way we "

# Volgende woorden genereren
output = generator(input_text, max_length=20, num_return_sequences=1)

# Print the predicted continuation
print("Voorspelde tekst:\n", output[0]["generated_text"])

Met een model kunnen we ook, naast de meest waarschijnlijke voorspelling, kijken naar net iets minder waarschijnlijke woorden. Dit kunnen je regelen met de zogenaamde 'temperatuur' instelling.

In [113]:
output = generator(input_text, max_length=20, num_return_sequences=3, temperature=0.8)
for i, result in enumerate(output):
    print(f"Prediction {i+1}: {result['generated_text']}")

Omdat de data waarop dit model is getrained, ongefilterde is vanop het internet, geeft de maker van deze dataset volgende waarschuwing:


*"Because large-scale language models like GPT-2 do not distinguish fact from fiction, **we don’t support use-cases that require the generated text to be true**.*
*Additionally, language models like GPT-2 reflect the biases inherent to the systems they were trained on, so we do not recommend that they be deployed into systems that interact with humans > unless the deployers first carry out a study of biases relevant to the intended use-case. We found no statistically significant difference in gender, race, and religious bias probes between 774M and 1.5B, implying all versions of GPT-2 should be approached with similar levels of caution around use cases that are sensitive to biases around human attributes.*

# LLM's

Larga language models zijn dus grote modellen die je kan gebruiken om tekst te voorspellen. Je kan zelf zulke getrainde modellen downloaden en op je eigen computer draaien. Je zal merken dat deze modellen veel trager reageren dan je gewend bent van bijvoorbeeld chatGPT. Dit is omdat jouw computer moeite heeft met de mijarden berekeningen die het voor elk woord te voorspellen, moet doen.

### Tekst genereren

Wanneer je 1 token kan voorspellen, is het een koud kunstje voor een computer om een heel antwoord te maken: je geeft bij een prompt door een gebruiker volgende tekst mee:

"
USER: "Hallo, chatGPT, kun je me iets vertellen over Napoleon?"
SYSTEM: "Ja, Napoleon is een Franse "
"

Bij elk volgend woord wordt deze input die je meegeeft aan de LLM iets langer, en op die manier produceert de machine tekst. Door de tekst van de gebruiker en die van de machine zelf duidelijk te coderen, krijg je iets wat op een conversatie lijkt.

### Langs een API, naar een model op internet

In [114]:
import requests
from dotenv import load_dotenv
import os
from mistralai import Mistral

# environment vars inladen
load_dotenv('secrets.env')

def ask_mistral_api(question):
    api_token = os.getenv("MISTRAL_TOKEN")
    model = "mistral-large-latest"

    client = Mistral(api_key=api_token)

    chat_response = client.chat.complete(
        model = model,
        messages = [
            {
                "role": "user",
                "content": question,
            },
        ]
    )
    return chat_response

# Example usage
question = "Wat is de hoofdstad van Frankrijk?"
answer = ask_mistral_api(question)
print(answer.choices[0].message.content)

### Lokaal, via Ollama
Als je deze demo zelf wil doen, zou je ollama moeten installeren op je systeem. Dit kan je thuis eens proberen ([www.ollama.ai](https://ollama.com/))
het verschil met de vorige code is dat dit model niet meer langs het internet gaat, maar enkel modellen lokaal op jouw computer gebruikt.

In [115]:
import ollama

De code hieronder zal beduidend langer duren dan de vorige, want je maakt nu geen gebruik meer van heel krachtige servers.

In [ ]:
response = ollama.chat(model='mistral', messages=[
  {
    'role': 'user',
    'content': 'Hoe werkt een regenboog?',
  },
])
print(response['message']['content'])